In [1]:
from numpy.core.numeric import True_
import sys
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.linear_model import LogisticRegression
#from matplotlib import pyplot as plt
#from PIL import Image, ImageDraw
#from IPython import display
#from torchvision import transforms

In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [3]:
data = 'heloc'
df = pd.read_csv('../datasets/'+data+'_sc.csv')

In [4]:
df.head()

,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MaxDelq2PublicRecLast12M,MaxDelqEver,...,NumTradesOpeninLast12M,PercentInstallTrades,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance,class
0,0.327586,0.177278,0.037736,0.333333,0.243590,0.157895,0.000000,0.7875,0.333333,0.500000,...,0.052632,0.450549,0.000000,0.000000,0.142241,0.25000,0.000000,0.055556,0.666667,0
1,0.534483,0.079900,0.047170,0.083333,0.102564,0.000000,0.000000,1.0000,0.777778,1.000000,...,0.210526,0.461538,0.060606,0.060606,0.228448,0.12500,0.045455,0.055556,0.849462,0
2,0.517241,0.208489,0.009434,0.287500,0.346154,0.052632,0.052632,0.9125,0.666667,0.666667,...,0.157895,0.604396,0.075758,0.060606,0.310345,0.18750,0.136364,0.166667,0.903226,0
3,0.775862,0.413233,0.254717,0.533333,0.141026,0.000000,0.000000,1.0000,0.777778,1.000000,...,0.000000,0.252747,0.015152,0.015152,0.219828,0.09375,0.000000,0.000000,0.784946,0
4,0.396552,0.168539,0.103774,0.308333,0.384615,0.000000,0.000000,0.8875,0.444444,0.666667,...,0.052632,0.494505,0.000000,0.000000,0.267241,0.37500,0.136364,0.166667,0.935484,0


In [5]:
target = 'class'
y = df[target].values
del df[target]
x = df.values

In [6]:
# train the classifier to be explained -> Logistic Regression Model (can be replaced with a MLP)
# clf = LogisticRegression(max_iter=1000, fit_intercept=False, class_weight='balanced')
# clf.fit(x, y)

In [7]:
# Convert numpy arrays to PyTorch tensors
x_train_tensor = torch.FloatTensor(x).to(device)
y_train_tensor = torch.LongTensor(y).to(device)

In [8]:
# Create a TensorDataset
dataset = TensorDataset(x_train_tensor, y_train_tensor)

In [9]:
# Create a DataLoader
dl_batch_size = 32  # Set your desired batch size
train_dataloader = DataLoader(dataset, batch_size=dl_batch_size, shuffle=True)

In [10]:
num_epochs = 500

In [11]:
x.shape[1]

20

In [12]:
latent_dims = 8
# capacity = no. of hidden neurons in a layer
capacity = 8
variational_beta = 1

In [13]:
latent_dims

8

In [14]:
class Encoder(nn.Module):
    def __init__(self, input_dim):
        super(Encoder, self).__init__()
        c = capacity
        self.fc1 = nn.Linear(in_features=input_dim, out_features=c*2)
        self.fc2 = nn.Linear(in_features=c*2, out_features=latent_dims)
        self.fc3 = nn.Linear(in_features=c*2, out_features=latent_dims)
            
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x_mu = self.fc2(x)
        x_logvar = self.fc3(x)
        return x_mu, x_logvar

class Decoder(nn.Module):
    def __init__(self, output_dim):
        super(Decoder, self).__init__()
        c = capacity
        self.fc = nn.Linear(in_features=latent_dims, out_features=c*2)
        self.fc_output = nn.Linear(in_features=c*2, out_features=output_dim)
            
    def forward(self, x):
        x = F.relu(self.fc(x))
        x_recon = self.fc_output(x)
        return x_recon

class VariationalAutoencoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = Encoder(input_dim)
        self.decoder = Decoder(output_dim)
    
    def forward(self, x):
        latent_mu, latent_logvar = self.encoder(x)
        latent = self.latent_sample(latent_mu, latent_logvar)
        x_recon = self.decoder(latent)
        return x_recon, latent_mu, latent_logvar
    
    def latent_sample(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = torch.empty_like(std).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu
    
def vae_loss(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x, x, reduction='sum')  # Mean Squared Error loss for tabular data
    kldivergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return recon_loss + variational_beta * kldivergence

In [15]:
# Train VAE
print(device)
vae = VariationalAutoencoder(input_dim=x.shape[1], output_dim=x.shape[1])
vae = vae.to(device)

optimizer = torch.optim.Adam(params=vae.parameters(), lr=1e-3, weight_decay=1e-5)

train_loss_avg = []

train_vae = True

if train_vae == False:   
    filename = 'blood_alcohol_vae.pth'
    vae.load_state_dict(torch.load(filename))
    vae.eval()
    print('done')

else:
    for epoch in range(num_epochs):
        vae.train()
        train_loss_avg.append(0)
        num_batches = 0

        for x_batch, _ in train_dataloader:

            x_batch = x_batch.to(device)

            # Forward pass
            x_recon, latent_mu, latent_logvar = vae(x_batch)
            loss = vae_loss(x_recon, x_batch, latent_mu, latent_logvar)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss_avg[-1] += loss.item()
            num_batches += 1

        train_loss_avg[-1] /= num_batches
        print('Epoch [%d / %d] average loss: %f' % (epoch + 1, num_epochs, train_loss_avg[-1]))

    # Save the model
    model_name = data+'_vae.pth'
    torch.save(vae.state_dict(), model_name)

mps
Epoch [1 / 500] average loss: 35.035088
Epoch [2 / 500] average loss: 14.298419
Epoch [3 / 500] average loss: 12.771334
Epoch [4 / 500] average loss: 12.333245
Epoch [5 / 500] average loss: 12.193708
Epoch [6 / 500] average loss: 12.123431
Epoch [7 / 500] average loss: 12.092136
Epoch [8 / 500] average loss: 12.051876
Epoch [9 / 500] average loss: 12.042833
Epoch [10 / 500] average loss: 12.037669
Epoch [11 / 500] average loss: 12.031599
Epoch [12 / 500] average loss: 12.017956
Epoch [13 / 500] average loss: 12.026116
Epoch [14 / 500] average loss: 12.030742
Epoch [15 / 500] average loss: 12.014887
Epoch [16 / 500] average loss: 12.013942
Epoch [17 / 500] average loss: 12.012345
Epoch [18 / 500] average loss: 12.016080
Epoch [19 / 500] average loss: 12.010727
Epoch [20 / 500] average loss: 12.017496
Epoch [21 / 500] average loss: 12.010641
Epoch [22 / 500] average loss: 12.005277
Epoch [23 / 500] average loss: 12.006335
Epoch [24 / 500] average loss: 12.013151
Epoch [25 / 500] aver

Epoch [199 / 500] average loss: 11.987343
Epoch [200 / 500] average loss: 11.992691
Epoch [201 / 500] average loss: 11.981523
Epoch [202 / 500] average loss: 11.984431
Epoch [203 / 500] average loss: 11.984717
Epoch [204 / 500] average loss: 11.984396
Epoch [205 / 500] average loss: 11.986941
Epoch [206 / 500] average loss: 11.978736
Epoch [207 / 500] average loss: 11.986231
Epoch [208 / 500] average loss: 11.989993
Epoch [209 / 500] average loss: 11.985216
Epoch [210 / 500] average loss: 11.981499
Epoch [211 / 500] average loss: 11.988517
Epoch [212 / 500] average loss: 11.979363
Epoch [213 / 500] average loss: 11.982372
Epoch [214 / 500] average loss: 11.989103
Epoch [215 / 500] average loss: 11.993363
Epoch [216 / 500] average loss: 11.984473
Epoch [217 / 500] average loss: 11.988492
Epoch [218 / 500] average loss: 11.989962
Epoch [219 / 500] average loss: 11.989671
Epoch [220 / 500] average loss: 11.993740
Epoch [221 / 500] average loss: 11.984177
Epoch [222 / 500] average loss: 11

Epoch [395 / 500] average loss: 11.972834
Epoch [396 / 500] average loss: 11.978733
Epoch [397 / 500] average loss: 11.974115
Epoch [398 / 500] average loss: 11.976470
Epoch [399 / 500] average loss: 11.975827
Epoch [400 / 500] average loss: 11.974205
Epoch [401 / 500] average loss: 11.975603
Epoch [402 / 500] average loss: 11.977354
Epoch [403 / 500] average loss: 11.971624
Epoch [404 / 500] average loss: 11.973277
Epoch [405 / 500] average loss: 11.981570
Epoch [406 / 500] average loss: 11.973862
Epoch [407 / 500] average loss: 11.972665
Epoch [408 / 500] average loss: 11.974536
Epoch [409 / 500] average loss: 11.971716
Epoch [410 / 500] average loss: 11.971183
Epoch [411 / 500] average loss: 11.973895
Epoch [412 / 500] average loss: 11.972382
Epoch [413 / 500] average loss: 11.978766
Epoch [414 / 500] average loss: 11.974639
Epoch [415 / 500] average loss: 11.969990
Epoch [416 / 500] average loss: 11.976046
Epoch [417 / 500] average loss: 11.972858
Epoch [418 / 500] average loss: 11

In [16]:
c2c_latent_dims = latent_dims-1
print(c2c_latent_dims)
c2c_capacity = latent_dims
print(c2c_capacity)
c2c_variational_beta = 1
label_size = 2  # Assuming two classes

7
8


In [17]:
class c2c_Encoder(nn.Module):
    def __init__(self):
        super(c2c_Encoder, self).__init__()
        self.fc1 = nn.Linear(in_features=latent_dims + label_size, out_features=c2c_capacity)
        self.fc_mu = nn.Linear(in_features=c2c_capacity, out_features=c2c_latent_dims)
        self.fc_logvar = nn.Linear(in_features=c2c_capacity, out_features=c2c_latent_dims)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x_mu = self.fc_mu(x)
        x_logvar = self.fc_logvar(x)
        return x_mu, x_logvar

class c2c_Decoder(nn.Module):
    def __init__(self):
        super(c2c_Decoder, self).__init__()
        self.fc2 = nn.Linear(in_features=c2c_latent_dims + label_size, out_features=c2c_capacity)
        self.fc1 = nn.Linear(in_features=c2c_capacity, out_features=latent_dims)
            
    def forward(self, x):
        x = self.fc2(x)
        x = self.fc1(x)
        return x
    
class c2c_VariationalAutoencoder(nn.Module):
    def __init__(self):
        super(c2c_VariationalAutoencoder, self).__init__()
        self.encoder = c2c_Encoder()
        self.decoder = c2c_Decoder()
    
    def forward(self, x):
        x,label = torch.split(x,(latent_dims,label_size),dim = 1)
        c2c_latent_mu, c2c_latent_logvar = self.encoder(torch.cat((x,label), dim=1))
        c2c_latent = self.latent_sample(c2c_latent_mu, c2c_latent_logvar)
        x_recon = self.decoder(torch.cat((c2c_latent,label), dim=1))
        return x_recon, c2c_latent_mu, c2c_latent_logvar
    
    def latent_sample(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = torch.empty_like(std).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu
    
def c2c_vae_loss(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x.view(-1, latent_dims), x.view(-1, latent_dims), reduction='sum')
    
    kldivergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return recon_loss + c2c_variational_beta * kldivergence

In [18]:
print(device)
c2c_vae = c2c_VariationalAutoencoder()
c2c_vae = c2c_vae.to(device)

optimizer = torch.optim.Adam(params=c2c_vae.parameters(), lr=1e-3, weight_decay=1e-5)

train_loss_avg = []

train_c2c_vae = True

if train_c2c_vae == False:
    filename = 'blood_alcohol_c2c.pth'
    c2c_vae.load_state_dict(torch.load(filename))
    c2c_vae.eval()
    print('done')
else:
    for epoch in range(num_epochs):
        c2c_vae.train()
        train_loss_avg.append(0)
        num_batches = 0

        for i in range(dl_batch_size):

            x_batch1, labels1 = next(iter(train_dataloader))
            x_batch2, labels2 = next(iter(train_dataloader))

            x_batch1 = x_batch1.to(device)
            x_batch_recon1, latent_mu1, latent_logvar1 = vae(x_batch1)

            x_batch2 = x_batch2.to(device)
            x_batch_recon2, latent_mu2, latent_logvar2 = vae(x_batch2) 

            latent_diff = latent_mu1 - latent_mu2 
            label_diff = torch.cat((labels1.unsqueeze(1), labels2.unsqueeze(1)), dim=1)
            #print(label_diff)

            latent_diff = latent_diff.to(device)
            label_diff = label_diff.float().to(device)

            c2c_input = torch.cat((latent_diff,label_diff), dim=1)
            c2c_input = c2c_input.to(device)

            c2c_recon, c2c_latent_mu, c2c_latent_logvar = c2c_vae(c2c_input)
            loss = c2c_vae_loss(c2c_recon, latent_diff.detach(), c2c_latent_mu, c2c_latent_logvar)
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            train_loss_avg[-1] += loss.item()
            num_batches += 1

        train_loss_avg[-1] /= num_batches
        print('Epoch [%d / %d] average reconstruction error: %f' % (epoch + 1, num_epochs, train_loss_avg[-1]))

    # Save the model
    model_name = data+'_c2c.pth'
    torch.save(c2c_vae.state_dict(), model_name)

mps
Epoch [1 / 500] average reconstruction error: 34.505207
Epoch [2 / 500] average reconstruction error: 17.806798
Epoch [3 / 500] average reconstruction error: 10.040385
Epoch [4 / 500] average reconstruction error: 5.961179
Epoch [5 / 500] average reconstruction error: 3.500201
Epoch [6 / 500] average reconstruction error: 2.216586
Epoch [7 / 500] average reconstruction error: 1.452948
Epoch [8 / 500] average reconstruction error: 1.002057
Epoch [9 / 500] average reconstruction error: 0.680750
Epoch [10 / 500] average reconstruction error: 0.474504
Epoch [11 / 500] average reconstruction error: 0.355284
Epoch [12 / 500] average reconstruction error: 0.263255
Epoch [13 / 500] average reconstruction error: 0.193728
Epoch [14 / 500] average reconstruction error: 0.145369
Epoch [15 / 500] average reconstruction error: 0.111702
Epoch [16 / 500] average reconstruction error: 0.088470
Epoch [17 / 500] average reconstruction error: 0.068716
Epoch [18 / 500] average reconstruction error: 0.0

Epoch [147 / 500] average reconstruction error: 0.001173
Epoch [148 / 500] average reconstruction error: 0.001254
Epoch [149 / 500] average reconstruction error: 0.001144
Epoch [150 / 500] average reconstruction error: 0.001086
Epoch [151 / 500] average reconstruction error: 0.001198
Epoch [152 / 500] average reconstruction error: 0.001124
Epoch [153 / 500] average reconstruction error: 0.001086
Epoch [154 / 500] average reconstruction error: 0.001098
Epoch [155 / 500] average reconstruction error: 0.001084
Epoch [156 / 500] average reconstruction error: 0.001161
Epoch [157 / 500] average reconstruction error: 0.001193
Epoch [158 / 500] average reconstruction error: 0.001089
Epoch [159 / 500] average reconstruction error: 0.001048
Epoch [160 / 500] average reconstruction error: 0.001152
Epoch [161 / 500] average reconstruction error: 0.001161
Epoch [162 / 500] average reconstruction error: 0.001157
Epoch [163 / 500] average reconstruction error: 0.001139
Epoch [164 / 500] average recon

Epoch [291 / 500] average reconstruction error: 0.001436
Epoch [292 / 500] average reconstruction error: 0.001304
Epoch [293 / 500] average reconstruction error: 0.001271
Epoch [294 / 500] average reconstruction error: 0.001282
Epoch [295 / 500] average reconstruction error: 0.001369
Epoch [296 / 500] average reconstruction error: 0.001444
Epoch [297 / 500] average reconstruction error: 0.001332
Epoch [298 / 500] average reconstruction error: 0.001236
Epoch [299 / 500] average reconstruction error: 0.001522
Epoch [300 / 500] average reconstruction error: 0.001194
Epoch [301 / 500] average reconstruction error: 0.001323
Epoch [302 / 500] average reconstruction error: 0.001429
Epoch [303 / 500] average reconstruction error: 0.001403
Epoch [304 / 500] average reconstruction error: 0.001316
Epoch [305 / 500] average reconstruction error: 0.001402
Epoch [306 / 500] average reconstruction error: 0.001419
Epoch [307 / 500] average reconstruction error: 0.001471
Epoch [308 / 500] average recon

Epoch [435 / 500] average reconstruction error: 0.001413
Epoch [436 / 500] average reconstruction error: 0.001234
Epoch [437 / 500] average reconstruction error: 0.001344
Epoch [438 / 500] average reconstruction error: 0.001254
Epoch [439 / 500] average reconstruction error: 0.001306
Epoch [440 / 500] average reconstruction error: 0.001315
Epoch [441 / 500] average reconstruction error: 0.001281
Epoch [442 / 500] average reconstruction error: 0.001204
Epoch [443 / 500] average reconstruction error: 0.001262
Epoch [444 / 500] average reconstruction error: 0.001380
Epoch [445 / 500] average reconstruction error: 0.001372
Epoch [446 / 500] average reconstruction error: 0.001359
Epoch [447 / 500] average reconstruction error: 0.001476
Epoch [448 / 500] average reconstruction error: 0.001297
Epoch [449 / 500] average reconstruction error: 0.001393
Epoch [450 / 500] average reconstruction error: 0.001355
Epoch [451 / 500] average reconstruction error: 0.001352
Epoch [452 / 500] average recon